<a href="https://colab.research.google.com/github/LiviaFanucchi/Introducao-ciencias-de-dados/blob/master/Aula_Tiago_25_11_Classificadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://grouplens.org/datasets/movielens/

Este Exemplo: https://grouplens.org/datasets/movielens/10m/

In [2]:
!pip install -U gitly

Requirement already up-to-date: gitly in /usr/local/lib/python3.6/dist-packages (1.1.4)


In [3]:
from gitly.colab.plot import GitlyPlotter

gitly = GitlyPlotter('git')
#gitly.config_render(renderer='colab')

In [4]:
import pandas as pd

In [10]:
df_filmes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ciencia de Dados/movies.dat', sep='::', names=['id_filmes', 'nomes', 'categorias'])
df_filmes.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,id_filmes,nomes,categorias
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df_rate = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ciencia de Dados/ratings.dat', sep='::', names=['id_usuario', 'id_filmes', 'nota', 'timestamp'])
df_rate.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,id_usuario,id_filmes,nota,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392


In [8]:
df_rate.id_usuario.nunique()

69875

# Estruturas

In [11]:
df_filmes.shape

(10681, 3)

In [12]:
df_rate.shape

(9999762, 4)

In [13]:
df_rate.id_filmes.value_counts()

296      34863
356      34457
593      33667
480      32629
318      31124
         ...  
64754        1
62237        1
7452         1
6085         1
27740        1
Name: id_filmes, Length: 10677, dtype: int64

In [15]:
df_rate.id_usuario.value_counts().rename('qtd_avaliadas')

59269    7359
67385    7047
14463    5169
68259    4483
27468    4449
         ... 
27513      20
56202      20
11120      20
17187      20
34554      20
Name: qtd_avaliadas, Length: 69875, dtype: int64

# Sumarização

In [14]:
import plotly.express as px

df = df_rate.id_usuario.value_counts().rename('qtd_avaliadas')
fig = px.histogram(df, x='qtd_avaliadas', marginal="box")
fig.show()

# S R - Inicio

In [16]:
df_rate.head()

,id_usuario,id_filmes,nota,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [17]:
# info 1 - somatoria da nota: Boa? (Não)
df_rate.groupby('id_usuario').sum()['nota'].head(8)

id_usuario
1     110.0
2      64.0
3     130.0
4     153.0
5     335.0
6     165.0
7     428.0
8    2718.5
Name: nota, dtype: float64

In [18]:
# info 2 - média da nota: Boa? (Sim)
df_rate.groupby('id_usuario').mean()['nota'].head(8)

id_usuario
1    5.000000
2    3.200000
3    3.939394
4    4.026316
5    3.850575
6    3.928571
7    3.926606
8    3.398125
Name: nota, dtype: float64

In [19]:
df_filmes.head()

,id_filmes,nomes,categorias
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
df_cat_exp = df_filmes.categorias.str.split('|', expand=True)
df_cat_exp.head(3)

,0,1,2,3,4,5,6,7
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None
1,Adventure,Children,Fantasy,None,None,None,None,None
2,Comedy,Romance,None,None,None,None,None,None


In [21]:
lista_genero = list(df_cat_exp[0].unique())
lista_genero.remove('IMAX')
lista_genero.remove('(no genres listed)')
print(lista_genero)

['Adventure', 'Comedy', 'Action', 'Animation', 'Drama', 'Crime', 'Sci-Fi', 'Children', 'Documentary', 'Mystery', 'Thriller', 'Horror', 'Fantasy', 'Western', 'Film-Noir', 'Romance', 'War', 'Musical']


In [22]:
voc_dict = {}

i=0
for gen in lista_genero:
    voc_dict[ gen.lower() ] = i
    i = i + 1
    
voc_dict

{'action': 2,
 'adventure': 0,
 'animation': 3,
 'children': 7,
 'comedy': 1,
 'crime': 5,
 'documentary': 8,
 'drama': 4,
 'fantasy': 12,
 'film-noir': 14,
 'horror': 11,
 'musical': 17,
 'mystery': 9,
 'romance': 15,
 'sci-fi': 6,
 'thriller': 10,
 'war': 16,
 'western': 13}

Usar o CountVectorizer para vetorizar o campo categoria do dataframe de filmes

Documentação: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Tutorial: https://kavita-ganesan.com/how-to-use-countvectorizer/#Example-of-How-CountVectorizer-Works

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vetorizador = CountVectorizer(  analyzer='word', vocabulary= voc_dict, ngram_range=(1,2), token_pattern = '[a-zA-Z0-9$&+,:;=?@#<>.^*()%!-]+' )
# Treinando o vetorizador
vetorizador.fit( lista_genero )

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None,
                token_pattern='[a-zA-Z0-9$&+,:;=?@#<>.^*()%!-]+',
                tokenizer=None,
                vocabulary={'action': 2, 'adventure': 0, 'animation': 3,
                            'children': 7, 'comedy': 1, 'crime': 5,
                            'documentary': 8, 'drama': 4, 'fantasy': 12,
                            'film-noir': 14, 'horror': 11, 'musical': 17,
                            'mystery': 9, 'romance': 15, 'sci-fi': 6,
                            'thriller': 10, 'war': 16, 'western': 13})

In [24]:
# Testando o vetorizador
matriz_cat_exemplo = vetorizador.transform(['Action|Crime|Thriller'])
matriz_cat_exemplo.todense()

matrix([[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [25]:
matriz_cat = vetorizador.transform( df_filmes.categorias )
matriz_cat.todense()[:5,:]

matrix([[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [26]:
df_filmes_cat_exp = pd.DataFrame(data=matriz_cat.todense(), columns=vetorizador.get_feature_names() )
df_filmes_cat_exp.head()

,adventure,comedy,action,animation,drama,crime,sci-fi,children,documentary,mystery,thriller,horror,fantasy,western,film-noir,romance,war,musical
0,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
df_filmes_cat = pd.merge( df_filmes, df_filmes_cat_exp, how='left', left_index=True, right_index=True )
df_filmes_cat.head()

,id_filmes,nomes,categorias,adventure,comedy,action,animation,drama,crime,sci-fi,children,documentary,mystery,thriller,horror,fantasy,western,film-noir,romance,war,musical
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
df_cat_exp = pd.merge( df_rate, df_filmes_cat, how='left', on='id_filmes' )
df_cat_exp.head(5)

,id_usuario,id_filmes,nota,timestamp,nomes,categorias,adventure,comedy,action,animation,drama,crime,sci-fi,children,documentary,mystery,thriller,horror,fantasy,western,film-noir,romance,war,musical
0,1,122,5.0,838985046,Boomerang (1992),Comedy|Romance,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,185,5.0,838983525,"Net, The (1995)",Action|Crime|Thriller,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2,1,231,5.0,838983392,Dumb & Dumber (1994),Comedy,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,292,5.0,838983421,Outbreak (1995),Action|Drama|Sci-Fi|Thriller,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
4,1,316,5.0,838983392,Stargate (1994),Action|Adventure|Sci-Fi,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
